In [64]:
import requests
import json
import time
import itertools
import os
from tqdm import tqdm
import sqlite3
import datetime
import pickle
import inspect
import string
import collections
import numpy as np
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha
from Artha.twitter import TwitterAPI
from Artha.sqlitedb import TSQLite
import Artha.configs.twitter_config as c

checkra = TwitterAPI(username = "checkra_", bearer_token = c.c_bearer, key = c.c_key, secret = c.c_secret, token = c.c_token, token_secret = c.c_token_secret)
print(checkra.id,"done")

1356259499431129092 done


In [2]:
checkra.user_lookup("BTC_JackSparrow")

{'id': '380546370', 'name': '//Bitcoin 𝕵ack 🐐', 'username': 'BTC_JackSparrow'}

In [3]:
tweets = checkra.get_recent_tweets("CryptoKaleo")
print(len(tweets))
tweet_text = [t["full_text"] for t in tweets]

2721


In [50]:
# code to get crypto tickers, save into tickers and names lists
with open("../data/crypto_tickers.json", "r") as w:
    tickers = json.loads(w.read())
too_common_tickers = ["ONE", "FOR", "TRY", "WIN", "HOT", "NEAR", "CAN", "HARD", "BULL", "SKY", "FUN", "EASY", "GO", "ADD", "OG", "DATA", "ATM", "ANT", "KEY"]
too_common_names = ["JUST", "HONEST"]
for i in too_common_tickers:
    del tickers[i]
for i in too_common_names:
    for k, v in tickers.items():
        if v == i:
            del tickers[k]
            break
coin_ticks = [tick.upper() for tick in tickers.keys()]
coin_names = [name.upper() for name in tickers.values()]

In [57]:
# methods to extract tickers, names, and cashtags for crypto

def extract_tickers(sentence):
    counts = {}
    sen = sentence.translate(str.maketrans('','',string.punctuation)).upper()
    sentence_words = sen.split()
    tickers = [word for word in sentence_words if word in coin_ticks]

    return collections.Counter(tickers)

def extract_names(sentence):

    sentence = " "+sentence.upper()+" "
    tickers = [coin for coin in coin_names if " "+coin+" " in sentence]
    return collections.Counter(tickers)

def extract_cashtags(sentence):
    words = sentence.upper().split()
    symbols = [word for word in words if len(word) > 1 and word[0]=="$" and word[1:].isalpha()]
    return collections.Counter(symbols)

# extract_general_tickers()
test = "I like BTC and ETH. but link sucks. I like $BTC. Bancor dope"
test2 = "Largest capitulation on bitcoin futures since March 2020"

In [61]:
# goes through entire set of recent tweets, extracts tickers
final_ticks = []
final_names = []
final_cash = []
with open("has.txt", "w+") as w, open("not.txt", "w+") as x:
    for ind, i in enumerate(tweet_text):
        tick = extract_tickers(i)
        name = extract_names(i)
        cash = extract_cashtags(i)
        final_ticks.extend(tick)
        final_names.extend(name)
        final_cash.extend([c[1:] for c in cash])
        if tick or name:
            w.write(i+"\n")
            json.dump(tick, w)
            w.write("\n")
            json.dump(name, w)
            w.write("\n\n\n")
        else:
            x.write(i+"\n\n")

In [62]:
print(set(final_ticks))

{'XRP', 'YFI', 'ETH', 'IOTA', 'SUSHI', 'AVAX', 'SC', 'RUNE', 'NPXS', 'EGLD', 'XLM', 'DOGE', 'BNB', 'SUN', 'NEO', 'CAKE', 'BOT', 'EOS', 'FUEL', 'DOT', 'ALPHA', 'REN', 'FTT', 'BAND', 'BLINK', 'XTZ', 'BTT', 'LTC', 'OMG', 'ENJ', 'FIL', 'GAS', 'TRX', 'LINK', 'USD', 'ZEC', 'SOL', 'PERP', 'SUB', 'DENT', 'SXP', 'CHZ', 'VIA', 'QTUM', 'ADA', 'USDT', 'BAL', '1INCH', 'BTC', 'ETC', 'BAKE', 'STRAX', 'SRM'}


In [63]:
print(set(final_cash))

{'XRP', 'PRAY', 'AAL', 'ETH', 'IOTA', 'NDX', 'OKB', 'BTMX', 'SPY', 'TSLA', 'AVAX', 'SC', 'NPXS', 'EGLD', 'OXY', 'XLM', 'DOGE', 'BNB', 'NEO', 'CAKE', 'RDD', 'EOS', 'PENN', 'HOT', 'DJI', 'FTT', 'BAND', 'XTZ', 'BTT', 'LTC', 'FTTCONTINUIJNG', 'OMG', 'ENJ', 'FIL', 'KIN', 'TRX', 'LINK', 'ZEC', 'SOL', 'DENT', 'COIN', 'SXP', 'QQQ', 'UAL', 'QTUM', 'ADA', 'BAL', 'BTC', 'ETC', 'STRAX', 'SRM', 'VIAC'}


In [66]:
np.setdiff1d(list(set(final_ticks)), list(set(final_cash)))

array(['1INCH', 'ALPHA', 'BAKE', 'BLINK', 'BOT', 'CHZ', 'DOT', 'FUEL',
       'GAS', 'PERP', 'REN', 'RUNE', 'SUB', 'SUN', 'SUSHI', 'USD', 'USDT',
       'VIA', 'YFI'], dtype='<U5')

In [67]:
np.setdiff1d(list(set(final_cash)), list(set(final_ticks)))

array(['AAL', 'BTMX', 'COIN', 'DJI', 'FTTCONTINUIJNG', 'HOT', 'KIN',
       'NDX', 'OKB', 'OXY', 'PENN', 'PRAY', 'QQQ', 'RDD', 'SPY', 'TSLA',
       'UAL', 'VIAC'], dtype='<U14')

In [ ]:
# TODO get better at extracting tickers from tweets
# TODO Download recent tweets for all people I follow